# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv()

True

In [3]:
sys.version

'3.13.5 (main, Jun 11 2025, 15:36:57) [Clang 17.0.0 (clang-1700.0.13.3)]'

## Create connection to AI Foundry Project & instantiate a project client

In [3]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Instantiate AGENT 1 (Q&A)

In [4]:


agent1 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Q&A",
    instructions="You are an AI agent.",
)
print(f"Created agent, agent ID: {agent1.id}")

Created agent, agent ID: asst_jKdkQAFhmPd5VLw2oixgS3Xa


## Instantiate AGENT 2 (Translator)

In [5]:


agent2 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Translator",
    instructions="You are an AI agent that translates English into Spanish.",
)
print(f"Created agent, agent ID: {agent2.id}")

Created agent, agent ID: asst_sZQs3HU51Fg9S5QKgaUVF1nm


## Instantiate AGENT 3 (Emojis creator)

In [6]:


agent3 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Emojis",
    instructions="You are an AI agent that add emojis to a text",
)
print(f"Created agent, agent ID: {agent3.id}")


Created agent, agent ID: asst_DgydlV9iEquVMgU00KwkUxWd


### Show all agents

In [7]:
print(agent1)
print(agent2)
print(agent3)


{'id': 'asst_jKdkQAFhmPd5VLw2oixgS3Xa', 'object': 'assistant', 'created_at': 1750257868, 'name': 'Q&A', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_sZQs3HU51Fg9S5QKgaUVF1nm', 'object': 'assistant', 'created_at': 1750257870, 'name': 'Translator', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that translates English into Spanish.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_DgydlV9iEquVMgU00KwkUxWd', 'object': 'assistant', 'created_at': 1750257872, 'name': 'Emojis', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


### Create a thread for AGENT 1

In [8]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_9F5jdRSM6E6AEwixRkc3q7EY


### Create a message for AGENT 1

In [ ]:

# [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content="Tell me about the 'Palacio de Bellas Artes' in Mexico City.")
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_BKnsB1gzXycowEVt5ugc3ZEH


### Run AGENT 1

In [10]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent1.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")


Created run, run ID: run_w4ek1kDE9xOKMIiOQOgjGDKa


### Check status of run for AGENT 1

In [11]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


### Show usage (tokens)

In [12]:
run.usage

{'prompt_tokens': 41, 'completion_tokens': 431, 'total_tokens': 472, 'prompt_token_details': {'cached_tokens': 0}}

### Show me the stack of messages so far

In [13]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")





Message ID: msg_U5QAs0jla1RYnTpj90cAQ2br, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': "The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of Mexico City's most iconic and significant cultural landmarks. Located in the historic center of Mexico City, the building is renowned for its striking blend of architectural styles and its role as the country's premier venue for the arts.\n\n### Architectural Features\n- **Design:** Construction began in 1904 under Italian architect Adamo Boari. It showcases an impressive blend of **Art Nouveau** (exterior) and **Art Deco** (interior) styles.\n- **Materials & Facade:** The exterior is clad in white Carrara marble, with ornate details and grand columns. The spectacular stained-glass curtain inside, designed by Tiffany & Co., depicts the Valley of Mexico and the volcanoes Popocatépetl and Iztaccíhuatl.\n- **Dome:** Its colorful tiled dome is an emblematic feature of Mexico City’s skyline.\n\n### Cultural Importanc

### Show last message of AGENT 1

In [14]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of Mexico City's most iconic and significant cultural landmarks. Located in the historic center of Mexico City, the building is renowned for its striking blend of architectural styles and its role as the country's premier venue for the arts.\n\n### Architectural Features\n- **Design:** Construction began in 1904 under Italian architect Adamo Boari. It showcases an impressive blend of **Art Nouveau** (exterior) and **Art Deco** (interior) styles.\n- **Materials & Facade:** The exterior is clad in white Carrara marble, with ornate details and grand columns. The spectacular stained-glass curtain inside, designed by Tiffany & Co., depicts the Valley of Mexico and the volcanoes Popocatépetl and Iztaccíhuatl.\n- **Dome:** Its colorful tiled dome is an emblematic feature of Mexico City’s skyline.\n\n### Cultural Importance\n- The Palacio is a central hub for the **performing arts** in Me

### SHow bottom of stack of messages

In [15]:
if messages:
    last_message = messages[-1]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Tell me about the 'Palacio de Bellas Artes' in Mexico City.", 'annotations': []}}]


### Run AGENT 2

In [16]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent2.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_HmLBCMdO8NUFaeJKbDvI3Eka


### Check Status of the run for AGENT 2

In [17]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [20]:
run.usage

{'prompt_tokens': 470, 'completion_tokens': 243, 'total_tokens': 713, 'prompt_token_details': {'cached_tokens': 0}}

### SHow the stack of messages so far

In [18]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")



Message ID: msg_TyC8qUpGkuKf2vNt5DPZ3HUC, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los monumentos más icónicos y representativos de la Ciudad de México. Ubicado en el centro histórico de la ciudad, es considerado el principal recinto cultural del país y una obra maestra tanto arquitectónica como artística.\n\nLa construcción del palacio comenzó en 1904 bajo la dirección del arquitecto italiano Adamo Boari, aunque se finalizó en 1934 debido a varios retrasos, incluyendo la Revolución Mexicana. El exterior del edificio es de estilo art nouveau y está recubierto de mármol blanco de Carrara, mientras que el interior exhibe un espléndido estilo art déco. Uno de sus elementos más notables es su cúpula de coloridos azulejos de vidrio.\n\nEn el Palacio de Bellas Artes se celebran eventos de danza, ópera, música y teatro. Alberga al Ballet Folklórico de México, la Orquesta Sinfónica Nacional y la Compañía Nacional de Ópera, 

### SHow last message of AGENT 2

In [19]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los monumentos más icónicos y representativos de la Ciudad de México. Ubicado en el centro histórico de la ciudad, es considerado el principal recinto cultural del país y una obra maestra tanto arquitectónica como artística.\n\nLa construcción del palacio comenzó en 1904 bajo la dirección del arquitecto italiano Adamo Boari, aunque se finalizó en 1934 debido a varios retrasos, incluyendo la Revolución Mexicana. El exterior del edificio es de estilo art nouveau y está recubierto de mármol blanco de Carrara, mientras que el interior exhibe un espléndido estilo art déco. Uno de sus elementos más notables es su cúpula de coloridos azulejos de vidrio.\n\nEn el Palacio de Bellas Artes se celebran eventos de danza, ópera, música y teatro. Alberga al Ballet Folklórico de México, la Orquesta Sinfónica Nacional y la Compañía Nacional de Ópera, entre otros destacados conjuntos artísticos. Además, es famoso por 

### Run AGENT 3

In [20]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent3.id)
 # [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_uX4qguHVU1qpAItOMuEALJOj


### Check status of run of AGENT 3

In [21]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.COMPLETED


In [22]:
run.usage

{'prompt_tokens': 792, 'completion_tokens': 172, 'total_tokens': 964, 'prompt_token_details': {'cached_tokens': 0}}

### Show the Stack of messages so far

In [23]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_CS3NVeISOaScD9ptppU0qknV, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'Of course! Here’s a version with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is one of Mexico City’s most iconic landmarks. Located in the historic downtown 🏙️, it’s famous for its beautiful white marble façade 🤍, stunning stained-glass dome 🎨, and impressive mix of architectural styles from Art Nouveau to Art Deco ✨.\n\nInside, you’ll find gorgeous murals by famous Mexican artists like Diego Rivera and Rufino Tamayo 🖼️. The palace hosts classical concerts, operas, ballets, and exhibitions 🎻🎤🩰, including the world-renowned Ballet Folklórico de México 💃.\n\nIt’s not just an architectural masterpiece but also a vibrant cultural center, making it a must-see spot when you visit Mexico City! 🇲🇽🌟', 'annotations': []}}]
Message ID: msg_TyC8qUpGkuKf2vNt5DPZ3HUC, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los mon

### Print last message of AGENT 3, this shows all Agents work combined in final output!!

In [24]:
messages = list(project_client.messages.list(thread_id=thread.id))

last_message = messages[0]
print(f"Content: {last_message.content}")




Content: [{'type': 'text', 'text': {'value': 'Of course! Here’s a version with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is one of Mexico City’s most iconic landmarks. Located in the historic downtown 🏙️, it’s famous for its beautiful white marble façade 🤍, stunning stained-glass dome 🎨, and impressive mix of architectural styles from Art Nouveau to Art Deco ✨.\n\nInside, you’ll find gorgeous murals by famous Mexican artists like Diego Rivera and Rufino Tamayo 🖼️. The palace hosts classical concerts, operas, ballets, and exhibitions 🎻🎤🩰, including the world-renowned Ballet Folklórico de México 💃.\n\nIt’s not just an architectural masterpiece but also a vibrant cultural center, making it a must-see spot when you visit Mexico City! 🇲🇽🌟', 'annotations': []}}]


## Post processing

In [25]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_DgydlV9iEquVMgU00KwkUxWd, Name: Emojis, Model: gpt-4.1, Instructions: You are an AI agent that add emojis to a text
Agent ID: asst_sZQs3HU51Fg9S5QKgaUVF1nm, Name: Translator, Model: gpt-4.1, Instructions: You are an AI agent that translates English into Spanish.
Agent ID: asst_jKdkQAFhmPd5VLw2oixgS3Xa, Name: Q&A, Model: gpt-4.1, Instructions: You are an AI agent.


In [27]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")
        break

In [28]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")